In [ ]:
# use deepmreye environment (python 3.9)
# pip install deepmreye 

# add this script into a folder structre as recommended by DeepMReye after cloning the repo 
# should contain model weights and functional data to be decoded

In [1]:
# Import modules and add library to path
import sys

sys.path.insert(0,"/Users/sinakling/projects/DeepMReye")
import os

os.environ[
    "CUDA_VISIBLE_DEVICES"] = ""  # Change to os.environ["CUDA_VISIBLE_DEVICES"] = "" if you dont have access to a GPU
import pickle

import numpy as np
import pandas as pd
# Initialize plotly figures
from plotly.offline import init_notebook_mode

# DeepMReye imports
from deepmreye import analyse, preprocess, train
from deepmreye.util import data_generator, model_opts

init_notebook_mode(connected=True)

# Make sure the output width is adjusted for better export as HTML
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:70% !important; }</style>"))
display(HTML("<style>.output_result { max-width:70% !important; }</style>"))

# Autoreload modules
%load_ext autoreload
%autoreload 2

/var/folders/v8/zmb9hj_97tjcq8mb570v8wq40000gp/T/ipykernel_23676/3227686564.py:23: DeprecationWarning:

Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display



In [2]:
# Define paths to functional data
experiment_folder = os.getcwd()
print(experiment_folder)
functional_data = os.path.join(experiment_folder, "functional_data")
processed_data = os.path.join(experiment_folder, "processed_data/")
model_weights = os.path.join(experiment_folder, "model_weights",
                             "datasets_1to6.h5")

# Create processed data folder if necessary
if not os.path.exists(processed_data):
    os.makedirs(processed_data)

# Get participants from functional folder
# (if needed, remove single participants with participants.remove('participant01') or recreate participants list)
participants = os.listdir(functional_data)

/Users/sinakling/projects/DeepMReye/calib_data


In [3]:
# Preload masks to save time within participant loop
(
    eyemask_small,
    eyemask_big,
    dme_template,
    mask,
    x_edges,
    y_edges,
    z_edges,
) = preprocess.get_masks()

# Loop across participants and extract eye mask
for participant in participants:

    if participant.startswith("s"):

        print(f"Running participant {participant}")
        participant_folder = os.path.join(functional_data, participant)

        for run in os.listdir(participant_folder):

            if run.startswith("sub"):

                # Filepath to functional
                fp_func = os.path.join(participant_folder, run)

                preprocess.run_participant(fp_func, dme_template, eyemask_big, eyemask_small, x_edges, y_edges, z_edges, transforms=['Affine', 'Affine', 'SyNAggro'])

                #preprocess.run_participant(
                #    fp_func,
                #    dme_template,
                #    eyemask_big,
                #    eyemask_small,
                #    x_edges,
                #    y_edges,
                #    z_edges,
                #)

Running participant sub-01
Mask 0/2, Sum: -37.182, Mean -3.099, Std 8.065, Median 0.004
Mask 1/2, Sum: 18.446, Mean 1.537, Std 5.682, Median 0.084
Mask 2/2, Sum: -10.846, Mean -0.904, Std 4.913, Median 0.014
Mask 0/2, Sum: -37.180, Mean -3.098, Std 8.093, Median -0.001
Mask 1/2, Sum: 11.818, Mean 0.985, Std 5.045, Median 0.028
Mask 2/2, Sum: -5.075, Mean -0.423, Std 4.342, Median 0.011
Mask 0/2, Sum: -37.429, Mean -3.119, Std 8.103, Median 0.003
Mask 1/2, Sum: 26.657, Mean 2.221, Std 6.572, Median 0.099
Mask 2/2, Sum: -16.374, Mean -1.365, Std 5.549, Median -0.001
Running participant sub-02
Mask 0/2, Sum: -23.187, Mean -1.932, Std 7.355, Median 0.047
Mask 1/2, Sum: 5.409, Mean 0.451, Std 5.713, Median 0.039
Mask 2/2, Sum: -0.246, Mean -0.021, Std 4.968, Median 0.080
Mask 0/2, Sum: -23.495, Mean -1.958, Std 7.362, Median 0.056
Mask 1/2, Sum: 9.853, Mean 0.821, Std 5.018, Median 0.094
Mask 2/2, Sum: -5.727, Mean -0.477, Std 4.677, Median -0.010
Mask 0/2, Sum: -21.559, Mean -1.797, Std 8.

In [4]:
# Combine processed masks with labels
for participant in participants:

    if participant.startswith("s"):

        print(f"Running participant {participant}")
        participant_folder = os.path.join(functional_data, participant)

        participant_data, participant_labels, participant_ids = [], [], []

        for run_idx, run in enumerate(os.listdir(participant_folder)):
            

            if not run.endswith(".p"):
                continue

            # Load mask and normalize it
            this_mask = os.path.join(participant_folder, run)
            this_mask = pickle.load(open(this_mask, "rb"))
            this_mask = preprocess.normalize_img(this_mask)

            # If experiment has no labels use dummy labels
            this_label = np.zeros(
                (this_mask.shape[3], 10, 2)
            )  # 10 is the number of subTRs used in the pretrained weights, 2 is XY

            # Check if each functional image has a corresponding label.
            # Note that mask has time as third dimension
            if this_mask.shape[3] != this_label.shape[0]:
                print(
                    f"WARNING --- Skipping Subject {participant} Run {run_idx} "
                    f"--- Wrong alignment (Mask {this_mask.shape} - Label {this_label.shape})."
                )

                continue

            # Store across runs
            participant_data.append(this_mask)  # adds data per run to list
            participant_labels.append(this_label)
            participant_ids.append(([participant] * this_label.shape[0],
                                    [run_idx] * this_label.shape[0]))
            

        # Save participant file
        preprocess.save_data(
            f"{participant}no_label",
            participant_data,
            participant_labels,
            participant_ids,
            processed_data,
            center_labels=False,
        )

Running participant sub-01


/Users/sinakling/projects/DeepMReye/deepmreye/preprocess.py:546: RuntimeWarning:

Mean of empty slice

/Users/sinakling/softwares/anaconda3/envs/deepmreye/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning:

Degrees of freedom <= 0 for slice.



Saving eye data (462, 47, 29, 18) and targets (462, 10, 2) (NaN 0) to file /Users/sinakling/projects/DeepMReye/calib_data/processed_data/sub-01no_label
Running participant sub-02
Saving eye data (462, 47, 29, 18) and targets (462, 10, 2) (NaN 0) to file /Users/sinakling/projects/DeepMReye/calib_data/processed_data/sub-02no_label


In [5]:
# Define paths to dataset
datasets = [
    processed_data + p for p in os.listdir(processed_data) if "no_label" in p
]

# Load data from one participant to showcase input/output
X, y = data_generator.get_all_subject_data(datasets[0])
print(f"Input: {X.shape}, Output: {y.shape}")

Input: (462, 47, 29, 18, 1), Output: (462, 10, 2)


In [ ]:
fig = analyse.visualise_input_data(X,
                                   y,
                                   bg_color="rgb(255,255,255)",
                                   ylim=[-11, 11])
fig.show()

In [6]:
opts = model_opts.get_opts()
test_participants = [
    processed_data + p for p in os.listdir(processed_data) if "no_label" in p
]
generators = data_generator.create_generators(test_participants,
                                              test_participants)
generators = (*generators, test_participants, test_participants
              )  # Add participant list

Training set (/Users/sinakling/projects/DeepMReye/calib_data/processed_data) contains 2 subjects: 
['sub-01no_label', 'sub-02no_label']
Test set (/Users/sinakling/projects/DeepMReye/calib_data/processed_data) contains 2 subjects: 
['sub-01no_label', 'sub-02no_label']


In [7]:
# Get untrained model and load with trained weights
(model, model_inference) = train.train_model(dataset="calib_data",
                                             generators=generators,
                                             opts=opts,
                                             return_untrained=True)
model_inference.load_weights(model_weights)

In [8]:
(evaluation, scores) = train.evaluate_model(
    dataset="calib_data",
    model=model_inference,
    generators=generators,
    save=False,
    model_path=experiment_folder,
    model_description="",
    verbose=2,
    percentile_cut=80,
)

/Users/sinakling/softwares/anaconda3/envs/deepmreye/lib/python3.9/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning:

invalid value encountered in divide

/Users/sinakling/softwares/anaconda3/envs/deepmreye/lib/python3.9/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning:

invalid value encountered in divide

/Users/sinakling/softwares/anaconda3/envs/deepmreye/lib/python3.9/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning:

invalid value encountered in divide

/Users/sinakling/softwares/anaconda3/envs/deepmreye/lib/python3.9/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning:

invalid value encountered in divide

/Users/sinakling/softwares/anaconda3/envs/deepmreye/lib/python3.9/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning:

invalid value encountered in divide

/Users/sinakling/softwares/anaconda3/envs/deepmreye/lib/python3.9/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning:

invalid value encountered in divide

1 / 2 - Model Performance for /Users/sinakling/projects/DeepMReye/calib_data/processed_data/sub-01no_label.npz
              Pearson          R^2-Score             Eucl. Error             
                    X   Y Mean         X     Y  Mean        Mean Median   Std
Default           NaN NaN  NaN     0.000 0.000 0.000       3.734  3.997 2.215
Default subTR     NaN NaN  NaN     0.000 0.000 0.000       3.718  3.990 2.213
Refined           NaN NaN  NaN     0.000 0.000 0.000       3.815  4.138 2.289
Refined subTR     NaN NaN  NaN     0.000 0.000 0.000       3.813  4.084 2.287


2 / 2 - Model Performance for /Users/sinakling/projects/DeepMReye/calib_data/processed_data/sub-02no_label.npz
              Pearson          R^2-Score             Eucl. Error             
                    X   Y Mean         X     Y  Mean        Mean Median   Std
Default           NaN NaN  NaN     0.000 0.000 0.000       3.219  3.694 1.784
Default subTR     NaN NaN  NaN     0.000 0.000 0.000       3.190  3.633 1.

/Users/sinakling/softwares/anaconda3/envs/deepmreye/lib/python3.9/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning:

invalid value encountered in divide

/Users/sinakling/softwares/anaconda3/envs/deepmreye/lib/python3.9/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning:

invalid value encountered in divide

/Users/sinakling/softwares/anaconda3/envs/deepmreye/lib/python3.9/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning:

invalid value encountered in divide

/Users/sinakling/softwares/anaconda3/envs/deepmreye/lib/python3.9/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning:

invalid value encountered in divide

/Users/sinakling/softwares/anaconda3/envs/deepmreye/lib/python3.9/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning:

invalid value encountered in divide

/Users/sinakling/softwares/anaconda3/envs/deepmreye/lib/python3.9/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning:

invalid value encountered in divide

In [9]:
def adapt_evaluation(participant_evaluation):
    pred_y = participant_evaluation["pred_y"]
    pred_y_median = np.nanmedian(pred_y, axis=1)
    pred_uncertainty = abs(participant_evaluation["euc_pred"])
    pred_uncertainty_median = np.nanmedian(pred_uncertainty, axis=1)
    df_pred_median = pd.DataFrame(
        np.concatenate(
            (pred_y_median, pred_uncertainty_median[..., np.newaxis]), axis=1),
        columns=["X", "Y", "Uncertainty"],
    )
    # With subTR
    subtr_values = np.concatenate((pred_y, pred_uncertainty[..., np.newaxis]),
                                  axis=2)
    index = pd.MultiIndex.from_product(
        [range(subtr_values.shape[0]),
         range(subtr_values.shape[1])],
        names=["TR", "subTR"])
    df_pred_subtr = pd.DataFrame(subtr_values.reshape(-1,
                                                      subtr_values.shape[-1]),
                                 index=index,
                                 columns=["X", "Y", "pred_error"])

    return df_pred_median, df_pred_subtr

In [10]:
# get evauluation into right format 

df_pred_median, df_pred_subtr = adapt_evaluation(evaluation[list(
            evaluation.keys())[0]])

In [24]:
# extract runs from sub TR resolution prediction

pred_sub_01_run_01_X = np.array(df_pred_subtr['X'][:1540])
pred_sub_01_run_02_X = np.array(df_pred_subtr['X'][1540:3080])
pred_sub_01_run_03_X = np.array(df_pred_subtr['X'][3080:])

pred_sub_01_run_01_Y = np.array(df_pred_subtr['Y'][:1540])
pred_sub_01_run_02_Y = np.array(df_pred_subtr['Y'][1540:3080])
pred_sub_01_run_03_Y = np.array(df_pred_subtr['Y'][3080:])

In [2]:
subject = 'sub-01'

In [4]:
expected_x = np.load(f'expected_x_calib_{subject}.npy')
expected_y = np.load(f'expected_y_calib_{subject}.npy')

expected_x_run_1 = expected_x[0]
expected_x_run_2 = expected_x[1]
expected_x_run_3 = expected_x[2]

expected_x_all_runs = [expected_x_run_1,expected_x_run_2,expected_x_run_3]

expected_y_run_1 = expected_y[0]
expected_y_run_2 = expected_y[1]
expected_y_run_3 = expected_y[2]

expected_y_all_runs = [expected_y_run_1,expected_y_run_2,expected_y_run_3]

In [5]:
eyetracking_x_run_1 = np.load(f'/Users/sinakling/projects/DeepMReye/calib_data/eyetracking_x_data_Calib_{subject}_run_1.npy')
eyetracking_x_run_2 = np.load(f'/Users/sinakling/projects/DeepMReye/calib_data/eyetracking_x_data_Calib_{subject}_run_2.npy')
eyetracking_x_run_3 = np.load(f'/Users/sinakling/projects/DeepMReye/calib_data/eyetracking_x_data_Calib_{subject}_run_3.npy')

eyetracking_all_runs_X = [eyetracking_x_run_1,eyetracking_x_run_2,eyetracking_x_run_3]

eyetracking_y_run_1 = np.load(f'/Users/sinakling/projects/DeepMReye/calib_data/eyetracking_y_data_Calib_{subject}_run_1.npy')
eyetracking_y_run_2 = np.load(f'/Users/sinakling/projects/DeepMReye/calib_data/eyetracking_y_data_Calib_{subject}_run_2.npy')
eyetracking_y_run_3 = np.load(f'/Users/sinakling/projects/DeepMReye/calib_data/eyetracking_y_data_Calib_{subject}_run_3.npy')

eyetracking_all_runs_Y = [eyetracking_y_run_1,eyetracking_y_run_2,eyetracking_y_run_3]

In [6]:
pred_sub_01_01_intpl_X = np.interp(
    np.linspace(0, 1, len(eyetracking_x_run_1)),
    np.linspace(0, 1, len(pred_sub_01_run_01_X)),
    pred_sub_01_run_01_X
)

pred_sub_01_02_intpl_X = np.interp(
    np.linspace(0, 1, len(eyetracking_x_run_2)),
    np.linspace(0, 1, len(pred_sub_01_run_02_X)),
    pred_sub_01_run_02_X
)

pred_sub_01_03_intpl_X = np.interp(
    np.linspace(0, 1, len(eyetracking_x_run_3)),
    np.linspace(0, 1, len(pred_sub_01_run_03_X)),
    pred_sub_01_run_03_X
)

predicted_all_runs_X = [pred_sub_01_01_intpl_X,pred_sub_01_02_intpl_X,pred_sub_01_03_intpl_X]

pred_sub_01_01_intpl_Y = np.interp(
    np.linspace(0, 1, len(eyetracking_y_run_1)),
    np.linspace(0, 1, len(pred_sub_01_run_01_Y)),
    -1.0*(pred_sub_01_run_01_Y)                   # Invert because of PsychToolBox coordinates
)

pred_sub_01_02_intpl_Y = np.interp(
    np.linspace(0, 1, len(eyetracking_y_run_2)),
    np.linspace(0, 1, len(pred_sub_01_run_02_Y)),
    -1.0*(pred_sub_01_run_02_Y)
)

pred_sub_01_03_intpl_Y = np.interp(
    np.linspace(0, 1, len(eyetracking_y_run_3)),
    np.linspace(0, 1, len(pred_sub_01_run_03_Y)),
    -1.0*(pred_sub_01_run_03_Y)
)

predicted_all_runs_Y = [pred_sub_01_01_intpl_Y,pred_sub_01_02_intpl_Y,pred_sub_01_03_intpl_Y]

In [16]:
np.save(f'deepmreye_X_{subject}_run_01',predicted_all_runs_X[0])
np.save(f'deepmreye_X_{subject}_run_02',predicted_all_runs_X[1])
np.save(f'deepmreye_X_{subject}_run_03',predicted_all_runs_X[2])


np.save(f'deepmreye_Y_{subject}_run_01',predicted_all_runs_Y[0])
np.save(f'deepmreye_Y_{subject}_run_02',predicted_all_runs_Y[1])
np.save(f'deepmreye_Y_{subject}_run_03',predicted_all_runs_Y[2])

In [ ]:
import scipy.stats as stats

eyetracking_all_runs_X_scaled = []

for elem in eyetracking_all_runs_X: 
    elem_scaled = stats.zscore(elem,nan_policy='omit')
    eyetracking_all_runs_X_scaled.append(elem_scaled)


expected_all_runs_X_scaled = []

for elem in expected_x_all_runs: 
    elem_scaled = stats.zscore(elem,nan_policy='omit')
    expected_all_runs_X_scaled.append(elem_scaled)




In [15]:
import scipy.stats as stats

predicted_all_runs_X_scaled = []

for elem in predicted_all_runs_X: 
    elem_scaled = stats.zscore(elem,nan_policy='omit')
    predicted_all_runs_X_scaled.append(elem_scaled)

np.save(f'deepmreye_X_{subject}_run_01',predicted_all_runs_X_scaled[0])
np.save(f'deepmreye_X_{subject}_run_02',predicted_all_runs_X_scaled[1])
np.save(f'deepmreye_X_{subject}_run_03',predicted_all_runs_X_scaled[2])

predicted_all_runs_Y_scaled = []

for elem in predicted_all_runs_Y: 
    elem_scaled = stats.zscore(elem,nan_policy='omit')
    predicted_all_runs_Y_scaled.append(elem_scaled) 

np.save(f'deepmreye_Y_{subject}_run_01',predicted_all_runs_Y_scaled[0])
np.save(f'deepmreye_Y_{subject}_run_02',predicted_all_runs_Y_scaled[1])
np.save(f'deepmreye_Y_{subject}_run_03',predicted_all_runs_Y_scaled[2])

In [7]:
# create dataframe 
# remove instructions time from beginning and end to make all same length
def trim_lists_to_shortest(input_lists):
    # Find the length of the shortest list
    min_length = min(len(lst) for lst in input_lists)

    # Trim each list to the length of the shortest one
    trimmed_lists = [lst[:min_length] for lst in input_lists]

    return trimmed_lists

eyetracking_data_all_runs_x_trim = trim_lists_to_shortest(eyetracking_all_runs_X)
eyetracking_data_all_runs_y_trim = trim_lists_to_shortest(eyetracking_all_runs_Y)


data = {"run_01_x": list(eyetracking_data_all_runs_x_trim[0]), "run_01_y": list(eyetracking_data_all_runs_y_trim[0]), "run_02_x": list(eyetracking_data_all_runs_x_trim[1]), "run_02_y": list(eyetracking_data_all_runs_y_trim[1]), "run_03_x": list(eyetracking_data_all_runs_x_trim[2]), "run_03_y": list(eyetracking_data_all_runs_y_trim[2])}

df = pd.DataFrame(data)
df

,run_01_x,run_01_y,run_02_x,run_02_y,run_03_x,run_03_y
0,-1.022457,-0.047424,-0.326276,0.039836,0.254192,-0.178314
1,-1.075572,0.256089,-0.322482,0.170726,0.254192,-0.178314
2,-1.075572,0.256089,-0.288337,0.299719,0.015176,-0.257986
3,-1.195080,-0.113817,-0.288337,0.299719,0.015176,-0.257986
4,-1.195080,-0.113817,-0.316791,0.326276,0.360421,-0.299719
...,...,...,...,...,...,...
184513,-0.240913,1.282340,0.092951,1.654143,1.030045,0.413536
184514,-0.239016,1.299413,-0.005691,1.671215,0.967446,0.421123
184515,-0.553910,1.337352,0.034145,1.589646,0.665830,0.375597
184516,-0.553910,1.337352,0.077775,1.591543,0.665830,0.375597


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


plot_rows = 3
plot_cols = 2

fig = make_subplots(rows=plot_rows, cols=plot_cols,shared_xaxes=True,vertical_spacing=0.05, subplot_titles= ['Hor. Coord. run 1', 'Ver. Coord. run 1', 'Hor. Coord. run 2', 'Ver. Coord. run 2', 'Hor. Coord. run 3', 'Ver. Coord. run 3'])

# Set a common y-axis range
common_y_range = [-15, 15]  # Adjust the range as needed

# add traces
x = 0
for i in range(1, plot_rows + 1):
    for j in range(1, plot_cols + 1):
        trace = go.Scatter(x=df.index, y=df[df.columns[x]].values,
                           name="Eyetracker Gaze Position",
                           mode='lines',
                           showlegend = True,
                           line=dict(color='#0E1C36', width=1.5))

        fig.add_trace(trace, row=i, col=j)

        
        # Hide legend for each subplot
        #fig.update_traces(showlegend=True, selector=dict(name= "Eyetracker Gaze Position"))

        # Set common Y-axis range
        fig.update_yaxes(range=common_y_range, row=i, col=j)

        x += 1

# add expected traces
#fig.add_trace(go.Scatter(y= expected_x_all_runs[0],showlegend=True, name='Expected Gaze Position',line=dict(color='#069D6B', width=2)), row = 1, col = 1)
#fig.add_trace(go.Scatter(y= expected_y_all_runs[0],showlegend=False, line=dict(color='#069D6B', width=2)), row = 1, col = 2)
#fig.add_trace(go.Scatter(y= expected_x_all_runs[1],showlegend=False,line=dict(color='#069D6B', width=2)), row = 2, col = 1)
#fig.add_trace(go.Scatter(y= expected_y_all_runs[1],showlegend=False,line=dict(color='#069D6B', width=2)), row = 2, col = 2)
#fig.add_trace(go.Scatter(y= expected_x_all_runs[2],showlegend=False,line=dict(color='#069D6B', width=2)), row = 3, col = 1)
#fig.add_trace(go.Scatter(y= expected_y_all_runs[2],showlegend=False,line=dict(color='#069D6B', width=2)), row = 3, col = 2)

# add predicted traces
fig.add_trace(go.Scatter(y= predicted_all_runs_X[0],showlegend=True, name='Predicted Gaze Position',line=dict(color='#6FA725', width=2)), row = 1, col = 1)
fig.add_trace(go.Scatter(y= predicted_all_runs_Y[0],showlegend=False, line=dict(color='#6FA725', width=2)), row = 1, col = 2)
fig.add_trace(go.Scatter(y= predicted_all_runs_X[1],showlegend=False,line=dict(color='#6FA725', width=2)), row = 2, col = 1)
fig.add_trace(go.Scatter(y= predicted_all_runs_Y[1],showlegend=False,line=dict(color='#6FA725', width=2)), row = 2, col = 2)
fig.add_trace(go.Scatter(y= predicted_all_runs_X[2],showlegend=False,line=dict(color='#6FA725', width=2)), row = 3, col = 1)
fig.add_trace(go.Scatter(y= predicted_all_runs_Y[2],showlegend=False,line=dict(color='#6FA725', width=2)), row = 3, col = 2)

##95B8D1

fig.add_vrect(x0="5400", x1="66500", 
                label=dict(
                text="fixation",
                textposition="top center"),
                fillcolor="#D8DDEF", opacity=0.2, line_width=0)

fig.add_vrect(x0="71000", x1="137000", 
                label=dict(
                text="pursuit",
                textposition="top center"),
                fillcolor="#A0A4B8", opacity=0.2, line_width=0)

fig.add_vrect(x0="141000", x1="179000", 
                label=dict(
                text="freeview",
                textposition="top center"),
                fillcolor="#7293A0", opacity=0.1, line_width=0)

# Update the legend to be not visible for specifc traces
fig.update_traces(showlegend= True, row = 1, col = 1, selector=dict(name= "Scanner Eyetracker Gaze Position"))
fig.update_traces(showlegend=False, row=1, col=2)
fig.update_traces(showlegend=False, row=2, col=1)
fig.update_traces(showlegend=False, row=2, col=2)
fig.update_traces(showlegend=False, row=3, col=1)
fig.update_traces(showlegend=False, row=3, col=2)



# Format and show fig
fig.update_layout(height=1200, width=3000, template="simple_white", title_text=f"{subject} Eyetracker Gaze Position vs. Expected Gaze Position vs. DeepMReye Untrained Predicted Gaze Position ", 
            yaxis1 = dict(title = "<b>Hor. coord. (dva)<b>", title_font=dict(size=12)),
            yaxis2 = dict(title = "<b>Ver. coord. (dva)<b>",title_font=dict(size=12)),
            yaxis3 = dict(title = "<b>Hor. coord. (dva)<b>",title_font=dict(size=12)),
            yaxis4 = dict(title = "<b>Ver. coord. (dva)<b>",title_font=dict(size=12)),
            xaxis5=dict(title='<b>Time (sec)<b>', tickmode = 'array', tickvals = [0, 20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000, 180000], ticktext = ['0', '20', "40", "60","80", "100", "120", "140", "160", "180"]),
            yaxis5 = dict(title = "<b>Hor. coord. (dva)<b>", title_font=dict(size=12)),
            xaxis6=dict(title='<b>Time (sec)<b>', tickmode = 'array', tickvals = [0, 20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000, 180000], ticktext = ['0', '20', "40", "60","80", "100", "120", "140", "160", "180"]),
            yaxis6 = dict(title = "<b>Ver. coord. (dva)<b>"))


# Update subplot titles font
fig.update_annotations(font=dict(size=14))


fig.show()
fig.write_image(f'{subject}_untrained_gazeposition_eyetracker.pdf')



In [ ]:
# difference/ error and additional plots in data_analysis_calib 